In [1]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import numpy as np
import imgaug.augmenters as iaa
import cv2
import statistics
import pandas as pd
import math

In [2]:
file = "catalina.png"
img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
img = np.float32(img)
cv2.imwrite("catalyna_gray.png", img)

True

In [3]:
def low_frequency(image, size=0.3):
    width = int(len(image)*size)
    height = int(len(image[0])*size)
    for i in range(len(image)):
        for j in range(len(image[i])):
            if i > width or j > height:
                image[i][j] = 0
    return image
            
function = low_frequency

In [4]:
def not_low_frequency(image, size=0.025):
    width = int(len(image)*size)
    height = int(len(image[0])*size)
    for i in range(width):
        for j in range(height):
            image[i][j] = 0
    return image

function = not_low_frequency

In [5]:
def dist(a, b, c = 5000):
    return math.sqrt(a*a+b*b)+c

def ratio_low_frequency(image, size=1):
    width = int(len(image)*size)
    height = int(len(image[0])*size)
    max_dist = dist(width, height)
    for i in range(len(image)):
        for j in range(len(image[i])):
            image[i][j] = image[i][j]*dist(i,j)/max_dist
    return image

function = ratio_low_frequency

In [6]:
def not_mid_frequency(image, size=0.9):
    width = int(len(image))
    height = int(len(image[0]))
    w_begin = int(width/2 - size*width/2)
    w_end = int(width/2 + size*width/2)
    h_begin = int(height/2 - size*height/2)
    h_end = int(height/2 + size*height/2)
    for i in range(w_begin, w_end):
        for j in range(h_begin, h_end):
            image[i][j] = 0
    return image

function = not_mid_frequency

In [7]:
e_file = "extra.png"
e_img = cv2.imread(e_file, cv2.IMREAD_GRAYSCALE)
e_img = np.float32(e_img)
e_dct = cv2.dct(e_img)

def two_images(image):
    width = int(len(image))
    height = int(len(image[0]))
    return np.float32([[0.25*e_dct[i][j] + 0.75*image[i][j] for j in range(height)] for i in range(width)])
            
function = two_images

In [8]:
def sample(image, size=0.1):
    pix = len(image)*len(image[0])
    mask = np.random.randint(100, size=(len(image), len(image[0])))
    return np.float32([[image[i][j] if mask[i][j] < 100*size else 0 for j in range(len(image[0]))] for i in range(len(image))])        

function = sample

In [203]:
dct = cv2.dct(img)
dct = function(dct)

cv2.imwrite("dtc.png", np.abs(dct))

True

In [197]:
inv_image = cv2.idct(dct)
inv_image = np.uint8(np.abs(inv_image))
cv2.imwrite("dtc_inv.png", inv_image)

True

In [23]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as spopt
import scipy.fftpack as spfft
import scipy.ndimage as spimg
import cvxpy as cvx

ModuleNotFoundError: No module named 'cvxpy'

In [24]:
def dct2(x):
    return spfft.dct(spfft.dct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

def idct2(x):
    return spfft.idct(spfft.idct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

In [28]:
from pylbfgs import owlqn

def evaluate(x, g, step):
    """An in-memory evaluation callback."""

    # we want to return two things: 
    # (1) the norm squared of the residuals, sum((Ax-b).^2), and
    # (2) the gradient 2*A'(Ax-b)

    # expand x columns-first
    x2 = x.reshape((nx, ny)).T

    # Ax is just the inverse 2D dct of x2
    Ax2 = idct2(x2)

    # stack columns and extract samples
    Ax = Ax2.T.flat[ri].reshape(b.shape)

    # calculate the residual Ax-b and its 2-norm squared
    Axb = Ax - b
    fx = np.sum(np.power(Axb, 2))

    # project residual vector (k x 1) onto blank image (ny x nx)
    Axb2 = np.zeros(x2.shape)
    Axb2.T.flat[ri] = Axb # fill columns-first

    # A'(Ax-b) is just the 2D dct of Axb2
    AtAxb2 = 2 * dct2(Axb2)
    AtAxb = AtAxb2.T.reshape(x.shape) # stack columns

    # copy over the gradient vector
    np.copyto(g, AtAxb)

    return fx

# fractions of the scaled image to randomly sample at
sample_sizes = (0.1, 0.01)

# read original image
Xorig = spimg.imread('catalina2.png')
ny,nx,nchan = Xorig.shape

# for each sample size
Z = [np.zeros(Xorig.shape, dtype='uint8') for s in sample_sizes]
masks = [np.zeros(Xorig.shape, dtype='uint8') for s in sample_sizes]
for i,s in enumerate(sample_sizes):

    # create random sampling index vector
    k = round(nx * ny * s)
    ri = np.random.choice(nx * ny, k, replace=False) # random sample of indices

    # for each color channel
    for j in range(nchan):

        # extract channel
        X = Xorig[:,:,j].squeeze()

        # create images of mask (for visualization)
        Xm = 255 * np.ones(X.shape)
        Xm.T.flat[ri] = X.T.flat[ri]
        masks[i][:,:,j] = Xm

        # take random samples of image, store them in a vector b
        b = X.T.flat[ri].astype(float)

        # perform the L1 minimization in memory
        Xat2 = owlqn(nx*ny, evaluate, None, 5)

        # transform the output back into the spatial domain
        Xat = Xat2.reshape(nx, ny).T # stack columns
        Xa = idct2(Xat)
        Z[i][:,:,j] = Xa.astype('uint8')

AttributeError: module 'scipy.ndimage' has no attribute 'imread'